# Dynamics Summary

This is a summary of the dynamics quantitites from a simulation presented in a way that is simple to check that the values are sensible before continuing with further analysis.

In [9]:
import pandas as pd
import altair.vegalite.v2 as alt
# This is for the interactive elements
import ipywidgets as widgets
from pathlib import Path
from sdanalysis.util import get_filename_vars
import matplotlib.pyplot as plt

# We are dealing with datsets larger than the 5000 rows,
# so save the data to a json file and read from that.
alt.data_transformers.enable('json');

The processing of the dataset has to be performed before this notebook will display anything useful. 
This processing can be performed using the command

```sh
make dynamics
```

in the root of this project directory.

In [10]:
# Reading the processed dataset which is generated by running `make dynamics`
df = pd.read_hdf('../data/analysis/dynamics_clean_agg.h5', 'dynamics')

# Process the dataset which involves
# - removing times <= 0 which don't work with log scales
# - Averaging over all the instances of start index
df = df.query('time > 0').groupby(
    ['temperature', 'pressure', 'time']
).mean().reset_index()

Since this notebook is focused on an interactive summary of the data,
the next cell selects every 5th value from the dataset.
This behaviour can be changed by changing the `every_nth` variable,
note however, that reducing this value will increase the time for a figure to load.

In [11]:
every_nth = 5
# Create a sub-dataset using only every 5th value
df_plot = df.groupby(['temperature', 'pressure']).nth(list(range(0, len(df), every_nth))).reset_index()

This is the part where the generation of the figure takes place.
The selection of the pressure and the dyanmic quantity is designed to be interactive,
selecting the value of interest.

In [12]:
pressures = widgets.ToggleButtons(description="Pressure", options=df.pressure.unique())

# The metadata columns, or those we are using for the other axes of the figure
# shouldn't be available for plotting since they are non-sensical.
metadata_cols = ["temperature", "pressure", "start_index", "time"]
axes = widgets.ToggleButtons(
    description="Quantity", 
    options=[col for col in df.columns  if col not in metadata_cols]
)

@widgets.interact(pressure=pressures, axis=axes)
def create_chart(pressure, axis):
    c = alt.Chart(df_plot.query('pressure==@pressure'), width=600, height=500).mark_point(size=75).encode(
        alt.X('time', axis=alt.Axis(format='e'), scale=alt.Scale(type='log')),
        alt.Y(axis, type='quantitative'),
        alt.Color('temperature:N')
    )
    # Set log log scale for displacement figures
    if axis in ['msd', 'mfd', 'mean_displacement']:
        c.encoding.y.scale = alt.Scale(type='log')
        
    return c

interactive(children=(ToggleButtons(description='Pressure', options=(1.0,), value=1.0), ToggleButtons(descript…

In [13]:
data_dir = "../data/simulations/dynamics/output/"
dataset = {}
for file in Path(data_dir).glob("thermo-*log"):
    temperature, pressure, _ = get_filename_vars(file.name)
    dataset.setdefault(pressure, {})
    dataset[pressure][temperature] = file

In [14]:
discard_quantities = ["timestep", "lz", "xz", "yz", "N"]
with open(file) as src:
    thermo_quantities = [c.strip() for c in src.readline().split('\t')]
    thermo_quantities = [c for c in thermo_quantities if c not in discard_quantities]
    thermo_quantities = [c for c in thermo_quantities if "rigid_center" not in c]

In [15]:
pressure = widgets.ToggleButtons(description="Pressure", options=list(dataset.keys()))
temperature = widgets.ToggleButtons(description="Temperature", options=dataset.get(pressure.value))
quantities = widgets.ToggleButtons(description="Quantity", options=thermo_quantities)

def update_temperatures(change):
    temperature.options = dataset.get(change.new)

pressure.observe(update_temperatures, names='value')


def plot_figure(filename, quantity):
    df = pd.read_csv(filename, sep='\t', index_col='timestep', usecols=["timestep", quantity])
    
    df = df[~df.index.duplicated(keep='first')].reset_index()
    df.index = pd.to_timedelta(df.timestep)
    df = df.resample('10ms').agg(['mean', 'std'])
    plt.errorbar(df.index.astype(int), df[quantity]['mean'], df[quantity]['std'], ecolor='k', elinewidth=1)
    
p = widgets.interactive(plot_figure, filename=temperature, quantity=quantities)

display(pressure, p)

ToggleButtons(description='Pressure', options=('1.00',), value='1.00')

interactive(children=(ToggleButtons(description='Temperature', options={'0.30': PosixPath('../data/simulations…